# Deploy and inference Reader LM v2 with Google Cloud Marketplace Virtual machine

This notebook demonstrates how to deploy a Reader LM V2 model-powered [Google Cloud Marketplace Virtual machine](https://console.cloud.google.com/marketplace/product/jinaai-public/reader-lm-v2) and perform inference with the application within the virtual machine.


## Deploy the managed application

To deploy the virtual machine, start by consulting the [official deployment guide](https://cloud.google.com/marketplace/docs/deploy-through-CLI#use-snippet). This document provides comprehensive steps for the deployment process.

It's worth mentioning that in the Basics tab of the deployment setup, you will need to provide several details about the VM. 

You can customize the VM size used, and for certain sizes, you might need to adjust the allowed quota to ensure access. It is recommended to select "GPUs" tab and use the [NVIDIA_L4](https://cloud.google.com/compute/docs/gpus?_gl=1*l4wato*_ga*MTI3NjQzMzcwNy4xNzI3NTg1MDYx*_ga_WH2QY8WWF5*MTczMDA4MDMzMC42NC4xLjE3MzAwODQ4NTQuMzcuMC4w#general_comparison_chart). This VM features up to 1 NVIDIA L4 GPU with 24 GB of memory. The boot disk size should be minimum 30 GB.

For the other tabs, you can leave most settings as default or adjust them to fit your needs.

<img src="images/deploy_reader_lm_v2.png" width="50%" height="50%" style="max-width: 600px">

Once the deployment of the VM is complete, proceed to the resource group created for your deployment to verify the resources that have been established. 

Within the resource group, you'll find the public IP through which you can access your application within the VM.

Please note that the application within the VM will be unavailable immediately after deployment due to necessary model loading process **We recommend waiting at least 7 minutes before using the application.**

# Perform inference with the managed application

First, construct the prompt using `create_prompt`, where you can specify the desired return format. This will improve the results according to specific scenarios.

In [ ]:
def create_prompt(text: str, return_type: str, instruction: str = None, schema: str = None):
    """
    Creates a prompt based on the specified return type (either 'json' or 'markdown').
    
    Parameters:
    - text (str): The HTML content that needs to be converted into the desired format.
    - return_type (str): The desired return format. It must be either "json" or "markdown".
    - instruction (str): The instruction to be included in the prompt. If not provided, a default instruction is used.
    - schema (str): The JSON schema for structuring the output (used only for 'json' return_type). If empty, no schema is included.

    """
    
    if return_type not in ["json", "markdown"]:
        raise ValueError("Invalid return_type. Must be 'json' or 'markdown'.")
    
    if return_type == "json":
        if not instruction:
            instruction = "Extract the main content from the given HTML and convert it in a structured JSON format."

        if schema:
            prompt = f"{instruction}\n```html\n{text}\n```\nThe JSON schema is as follows:```json{schema}```" 
        else:
            prompt = f"{instruction}\n```html\n{text}\n```"
    
    elif return_type == "markdown":
        if not instruction:
            instruction = "Extract the main content from the given HTML and convert it to Markdown format."
        prompt = f"{instruction}\n```html\n{text}\n```"
    
    return prompt

html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Phone Book</title>
</head>
<body>
    <h1>Phone Book</h1>
    <div class="contact">
        <h2>John Doe</h2>
        <p>Email: <a href="mailto:john.doe@example.com">john.doe@example.com</a></p>
    </div>
    <div class="contact">
        <h2>Jane Smith</h2>
        <p>Email: <a href="mailto:jane.smith@example.com">jane.smith@example.com</a></p>
    </div>
</body>
</html>
"""

prompt = create_prompt(return_type="markdown", text=html)
print(prompt)

Then create the invoke function to execute inference, shown as below:

In [ ]:
import json

import requests


def invoke_endpoint():
    url = "http://<Insert here your public IP address>/invocations"  # With above example, it's "http://20.84.48.180/invocations"
    headers = {"Content-Type": "application/json"}
    json_data = {
        "model": "ReaderLM-v2",
        "prompt": prompt,
        "stream": False  # Whether to stream back partial progress.
    }

    response = requests.post(url, headers=headers, data=json.dumps(json_data))
    print(response.json())


invoke_endpoint()